In [1]:
import pandas as pd

In [49]:
fpath = "Resources/purchase_data.csv"
hop_df = pd.read_csv(fpath,encoding = 'latin1')
hop_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Variables

In [20]:
player_count = 0
item_count = 0
ave_purch = str()

## Player Count

In [14]:
player_count = hop_df['SN'].nunique()
player_count

576

## Purchasing Analysis (Total)
* Number Unique Items
* Average Purchase Price
* Total Number of Purchases
* Total Revenue

In [54]:
item_count = hop_df['Item Name'].nunique()
print(f'Number of Unique Items:{item_count}')

ave_purch = hop_df['Price'].mean()
print(f'Average Purchase Price: ${ave_purch:.2f}')

tot_purch_count = hop_df['Purchase ID'].nunique()
print(f'Total Number of Purchases: {tot_purch_count}')

tot_rev = hop_df['Price'].sum()
print(f'Total Revenue: ${tot_rev:.2f}')

Number of Unique Items:179
Average Purchase Price: $3.05
Total Number of Purchases: 780
Total Revenue: $2379.77


### Gender Demographics

* Percentage and Count of Male Players
* Percentage and Count of Female Players
* Percentage and Count of Other / Non-Disclosed

### Purchasing Analysis (Gender)

* The below each broken by gender
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value
  * Average Purchase Total per Person by Gender

### Age Demographics

* The below each broken into bins of 4 years (i.e. &lt;10, 10-14, 15-19, etc.)
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value
  * Average Purchase Total per Person by Age Group

### Purchasing Analysis (age)

* The below each broken into bins of 4 years (i.e. &lt;10, 10-14, 15-19, etc.)
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value
  * Average Purchase Total per Person by Age

### Top Spenders

* Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
  * SN
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value

### Most Popular Items

* Identify the 5 most popular items by purchase count, then list (in a table):
  * Item ID
  * Item Name
  * Purchase Count
  * Item Price
  * Total Purchase Value

### Most Profitable Items

* Identify the 5 most profitable items by total purchase value, then list (in a table):
  * Item ID
  * Item Name
  * Purchase Count
  * Item Price
  * Total Purchase Value